# Introduction to Natural Lenguage processing in Pytorch

## Word embedding
Word embeddings, or word vectors, provide a way of mapping words from a vocabulary into a low-dimensional space, where words with similar meanings are close together.

Read *[Introduction to Word Embedding and Word2Vec][intro to word]*

> Our objective is to have words with similar context occupy close spatial positions. Mathematically, the cosine of the angle between such vectors should be close to 1, i.e. angle close to 0.
<br><br>Here comes the idea of generating distributed representations. Intuitively, we introduce some dependence of one word on the other words. The words in context of this word would get a greater share of this dependence. In one hot encoding representations, all the words are independent of each other

![Representation of difference between words](https://miro.medium.com/v2/resize:fit:720/format:webp/0*XMW5mf81LSHodnTi.png)

[intro to word]:[https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa]

In [13]:
# Download word vectors
from urllib.request import urlretrieve
import os
if not os.path.isfile('datasets/mini.h5'):
    print("Downloading Conceptnet Numberbatch word embeddings...")
    conceptnet_url = 'http://conceptnet.s3.amazonaws.com/precomputed-data/2016/numberbatch/17.06/mini.h5'
    urlretrieve(conceptnet_url, 'datasets/mini.h5')
    print("Downloading finished")
print("Dataset in memory")

Dataset in memory


To read an h5 file, we'll need to use the h5py package. If you followed the PyTorch installation instructions in 1A, you should have it downloaded already. Otherwise, you can install it with

If you environment isn't currently active, activate it: `conda activate pytorch` 
then `pip install h5py`


You may need to re-open this notebook for the installation to take effect.
Below, we use the package to open the mini.h5 file we just downloaded. We extract from the file a list of utf-8-encoded words, as well as their 300-d vectors.

In [14]:
# Load the file and pull out words and embeddings
import h5py

with h5py.File('datasets/mini.h5', 'r') as f:
    all_words = [word.decode('utf-8') for word in f['mat']['axis1'][:]]
    all_embeddings = f['mat']['block0_values'][:]
    
print("all_words dimensions: {}".format(len(all_words)))
print("all_embeddings dimensions: {}".format(all_embeddings.shape))

print("Random example word: {}".format(all_words[1337]))

all_words dimensions: 362891
all_embeddings dimensions: (362891, 300)
Random example word: /c/de/aufmachung


Now, `all_words` is a list of $V$ strings (what we call our *vocabulary*), and `all_embeddings` is a $V \times 300$ matrix. The strings are of the form `/c/language_code/word`—for example, `/c/en/cat` and `/c/es/gato`.

We are interested only in the English words. We use Python list comprehensions to pull out the indices of the English words, then extract just the English words (stripping the six-character `/c/en/` prefix) and their embeddings

In [15]:
# Restrict our vocabulary to just the English words
english_words = [word[6:] for word in all_words if word.startswith('/c/en/')]
english_word_indices = [i for i, word in enumerate(all_words) if word.startswith('/c/en/')]
english_embeddings = all_embeddings[english_word_indices]

print("Number of English words in all_words: {0}".format(len(english_words)))
print("english_embeddings dimensions: {0}".format(english_embeddings.shape))

print(english_words[1337])

Number of English words in all_words: 150875
english_embeddings dimensions: (150875, 300)
activated_carbon


The magnitude of a word vector is less important than its direction; the magnitude can be thought of as representing frequency of use, independent of the semantics of the word. 
Here, we will be interested in semantics, so we *normalize* our vectors, dividing each by its length. 
The result is that all of our word vectors are length 1, and as such, lie on a unit circle. 
The dot product of two vectors is proportional to the cosine of the angle between them, and provides a measure of similarity (the bigger the cosine, the smaller the angle).

<img src="https://images.deepai.org/glossary-terms/cosine-similarity-1007790.jpg" alt="cosine" style="max-width: 1200px;"/>

In [16]:
import numpy as np

norms = np.linalg.norm(english_embeddings, axis=1)
normalized_embeddings = english_embeddings.astype('float32') / norms.astype('float32').reshape([-1, 1])

We want to look up words easily, so we create a dictionary that maps us from a word to its index in the word embeddings matrix.

In [17]:
index = {word: i for i, word in enumerate(english_words)}

Now we are ready to measure the similarity between pairs of words. We use numpy to take dot products.

In [18]:
def similarity_score(w1, w2):
    score = np.dot(normalized_embeddings[index[w1], :], normalized_embeddings[index[w2], :])
    return score

# A word is as similar with itself as possible:
print('cat\tcat\t', similarity_score('cat', 'cat'))

# Closely related words still get high scores:
print('cat\tfeline\t', similarity_score('cat', 'feline'))
print('cat\tdog\t', similarity_score('cat', 'dog'))

# Unrelated words, not so much
print('cat\tmoo\t', similarity_score('cat', 'moo'))
print('cat\tfreeze\t', similarity_score('cat', 'freeze'))

# Antonyms are still considered related, sometimes more so than synonyms
print('antonym\topposite\t', similarity_score('antonym', 'opposite'))
print('antonym\tsynonym\t', similarity_score('antonym', 'synonym'))

cat	cat	 1.0000001
cat	feline	 0.8199548
cat	dog	 0.590724
cat	moo	 0.0039538303
cat	freeze	 -0.030225191
antonym	opposite	 0.3941065
antonym	synonym	 0.46883982


We can also find, for instance, the most similar words to a given word.

In [19]:
def closest_to_vector(v, n):
    all_scores = np.dot(normalized_embeddings, v)
    best_words = list(map(lambda i: english_words[i], reversed(np.argsort(all_scores))))
    return best_words[:n]

def most_similar(w, n):
    return closest_to_vector(normalized_embeddings[index[w], :], n)

In [20]:
print(most_similar('cat', 10))
print(most_similar('dog', 10))
print(most_similar('duke', 10))

['cat', 'humane_society', 'kitten', 'feline', 'colocolo', 'cats', 'kitty', 'maine_coon', 'housecat', 'sharp_teeth']
['dog', 'dogs', 'wire_haired_dachshund', 'doggy_paddle', 'lhasa_apso', 'good_friend', 'puppy_dog', 'bichon_frise', 'woof_woof', 'golden_retrievers']
['duke', 'dukes', 'duchess', 'duchesses', 'ducal', 'dukedom', 'duchy', 'voivode', 'princes', 'prince']


We can also use `closest_to_vector` to find words "nearby" vectors that we create ourselves. This allows us to solve analogies. For example, in order to solve the analogy "man : brother :: woman : ?", we can compute a new vector `brother - man + woman`: the meaning of brother, minus the meaning of man, plus the meaning of woman. We can then ask which words are closest, in the embedding space, to that new vector.

In [21]:
def solve_analogy(a1, b1, a2):
    b2 = normalized_embeddings[index[b1], :] - normalized_embeddings[index[a1], :] + normalized_embeddings[index[a2], :]
    return closest_to_vector(b2, 1)

print(solve_analogy("man", "brother", "woman"))
print(solve_analogy("man", "husband", "woman"))
print(solve_analogy("spain", "madrid", "france"))

['sister']
['wife']
['paris']


These three results are quite good, but in general, the results of these analogies can be disappointing. Try experimenting with other analogies, and see if you can think of ways to get around the problems you notice (i.e., modifications to the `solve_analogy()` algorithm).

### Using word embeddings in deep models

Word embeddings are fun to play around with, but their primary use is that they allow us to think of words as existing in a continuous, Euclidean space; we can then use an existing arsenal of techniques for machine learning with continuous numerical data (like logistic regression or neural networks) to process text.
Let's take a look at an especially simple version of this. 
We'll perform *sentiment analysis* on a set of movie reviews: in particular, we will attempt to classify a movie review as positive or negative based on its text.

We will use a [Simple Word Embedding Model](http://people.ee.duke.edu/~lcarin/acl2018_swem.pdf) (SWEM, Shen et al. 2018) to do so. 
We will represent a review as the *mean* of the embeddings of the words in the review. 
Then we'll train a two-layer MLP (a neural network) to classify the review as positive or negative.
As you might guess, using just the mean of the embeddings discards a lot of the information in a sentences, but for tasks like sentiment analysis, it can be surprisingly effective.

If you don't have it already, download the `movie-simple.txt` file. 
Each line of that file contains 

1. the numeral 0 (for negative) or the numeral 1 (for positive), followed by
2. a tab (the whitespace character), and then
3. the review itself.

Let's first read the data file, parsing each line into an input representation and its corresponding label.
Again, since we're using SWEM, we're going to take the mean of the word embeddings for all the words as our input.

In [33]:
import string
remove_punct=str.maketrans('','',string.punctuation)

# This function converts a line of our data file into
# a tuple (x, y), where x is 300-dimensional representation
# of the words in a review, and y is its label.
def convert_line_to_example(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [normalized_embeddings[index[w]] for w in words
                  if w in index]
    
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return x, y

# Apply the function to each line in the file.
xs = []
ys = []
with open("movie-simple.txt", "r", encoding='utf-8', errors='ignore') as f:
    for l in f.readlines():
        x, y = convert_line_to_example(l)
        xs.append(x)
        ys.append(y)
        
# Concatenate all examples into a numpy array
xs = np.vstack(xs)
ys = np.vstack(ys)

In [34]:
print("Shape of inputs: {}".format(xs.shape))
print("Shape of labels: {}".format(ys.shape))

num_examples = xs.shape[0]

Shape of inputs: (1411, 300)
Shape of labels: (1411, 1)


Notice that with this set-up, our input words have been converted to vectors as part of our preprocessing. This essentially locks our word embeddings in place throughout training, as opposed to learning the word embeddings. Learning word embeddings, either from scratch or fine-tuned from some pre-trained initialization, is often desirable, as it specializes them for the specific task. However, because our data set is relatively small and our computation budget for this demo, we're going to forgo learning the word embeddings for this model. We'll revist this in a bit.

Now that we've parsed the data, let's save 20% of the data (rounded to a whole number) for testing, using the rest for training. The file we loaded had all the negative reviews first, followed by all the positive reviews, so we need to shuffle it before we split it into the train and test splits. We'll then convert the data into PyTorch Tensors so we can feed them into our model.

In [41]:
print("First 20 labels before shuffling: {0}".format(ys[:20, 0]))

shuffle_idx = np.random.permutation(num_examples)
xs = xs[shuffle_idx, :]
ys = ys[shuffle_idx, :]

print("First 20 labels after shuffling: {0}".format(ys[:20, 0]))

First 20 labels before shuffling: [1 1 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1]
First 20 labels after shuffling: [1 0 1 0 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0]


In [46]:
import torch

num_train = 4*num_examples // 5

x_train = torch.tensor(xs[:num_train])
y_train = torch.tensor(ys[:num_train], dtype=torch.float32)

x_test = torch.tensor(xs[num_train:])
y_test = torch.tensor(ys[num_train:], dtype=torch.float32)

We could format each batch individually as we feed it into the model, but to make it easier on ourselves, let's create a TensorDataset and DataLoader as we've used in the past for MNIST.

In [48]:
reviews_train = torch.utils.data.TensorDataset(x_train, y_train)
reviews_test = torch.utils.data.TensorDataset(x_test, y_test)

train_loader = torch.utils.data.DataLoader(reviews_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(reviews_test, batch_size=100, shuffle=False)

## Building model in PyTorch

In [49]:
import torch.nn as nn
import torch.nn.functional as F

First we build the model, organized as a nn.Module. We could make the number of outputs for our MLP the number of classes for this dataset (i.e. 2). However, since we only have two output classes here ("positive" vs "negative"), we can instead produce a single output value, calling everything greater than  0
  "postive" and everything less than  0
  "negative". If we pass this output through a sigmoid operation, then values are mapped to  [0,1]
 , with  0.5
  being the classification threshold.

In [50]:
class SWEM(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

To train the model, we instantiate the model. 
Notice that since we are only doing binary classification, we use the binary cross-entropy (BCE) loss instead of the cross-entropy loss we've seen before.
We use the "with logits" version for numerical stability.

In [51]:
## Training
# Instantiate model
model = SWEM()

# Binary cross-entropy (BCE) Loss and Adam Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Iterate through train set minibatchs 
for epoch in range(250):
    correct = 0
    num_examples = 0
    for inputs, labels in train_loader:
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        y = model(inputs)
        loss = criterion(y, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        predictions = torch.round(torch.sigmoid(y))
        correct += torch.sum((predictions == labels).float())
        num_examples += len(inputs)
    
    # Print training progress
    if epoch % 25 == 0:
        acc = correct/num_examples
        print("Epoch: {0} \t Train Loss: {1} \t Train Acc: {2}".format(epoch, loss, acc))

## Testing
correct = 0
num_test = 0

with torch.no_grad():
    # Iterate through test set minibatchs 
    for inputs, labels in test_loader:
        # Forward pass
        y = model(inputs)
        
        predictions = torch.round(torch.sigmoid(y))
        correct += torch.sum((predictions == labels).float())
        num_test += len(inputs)
    
print('Test accuracy: {}'.format(correct/num_test))

Epoch: 0 	 Train Loss: 0.6779959797859192 	 Train Acc: 0.542553186416626
Epoch: 25 	 Train Loss: 0.14540649950504303 	 Train Acc: 0.9494680762290955
Epoch: 50 	 Train Loss: 0.09951790422201157 	 Train Acc: 0.9689716100692749
Epoch: 75 	 Train Loss: 0.07435806095600128 	 Train Acc: 0.9796099066734314
Epoch: 100 	 Train Loss: 0.08585225045681 	 Train Acc: 0.9840425252914429
Epoch: 125 	 Train Loss: 0.052023258060216904 	 Train Acc: 0.9884752035140991
Epoch: 150 	 Train Loss: 0.021740058436989784 	 Train Acc: 0.991134762763977
Epoch: 175 	 Train Loss: 0.016494380310177803 	 Train Acc: 0.993794322013855
Epoch: 200 	 Train Loss: 0.011507573537528515 	 Train Acc: 0.9973404407501221
Epoch: 225 	 Train Loss: 0.01794351078569889 	 Train Acc: 0.9973404407501221
Test accuracy: 0.9257950782775879


In [52]:
# Check some words
words_to_test = ["exciting", "hated", "boring", "loved"]

for word in words_to_test:
    x = torch.tensor(normalized_embeddings[index[word]].reshape(1, 300))
    print("Sentiment of the word '{0}': {1}".format(word, torch.sigmoid(model(x))))

Sentiment of the word 'exciting': tensor([[1.]], grad_fn=<SigmoidBackward0>)
Sentiment of the word 'hated': tensor([[1.9657e-27]], grad_fn=<SigmoidBackward0>)
Sentiment of the word 'boring': tensor([[2.5549e-20]], grad_fn=<SigmoidBackward0>)
Sentiment of the word 'loved': tensor([[1.]], grad_fn=<SigmoidBackward0>)


Try some words of your own!
You can also try changing up the model and re-training it to see how the results change.
Can you modify the architecture to get better performance?
Alternatively, can you simplify the model without sacrificing too much accuracy?
What if you try to classify the mean embeddings directly?